In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np

xg = pd.read_csv('./pred_probs/xgboost_pred_prob.csv').values
xg3 = pd.read_csv('./pred_probs/3xgboost_prob.csv').values
wxg = pd.read_csv('./pred_probs/w_xgboost_prob.csv').values

# STEP 1: Stack predictions as meta-features
X_meta_train = np.hstack([xg, xg3, wxg])
X_meta_test = np.hstack([xg, xg3, wxg])

# STEP 2: Train meta-classifier
X_tr, X_val, y_tr, y_val = train_test_split(X_meta_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

meta_model = LogisticRegression(multi_class='multinomial', max_iter=1000)
meta_model.fit(X_tr, y_tr)

# STEP 3: Predict on test
meta_probs = meta_model.predict_proba(X_meta_test)

# STEP 4: Get top-3 labels
top3_indices = np.argsort(meta_probs, axis=1)[:, -3:][:, ::-1]
top3_labels = [' '.join(label_encoder.classes_[i] for i in row) for row in top3_indices]
